<a href="https://colab.research.google.com/github/georgiam11/mcap-trading-project/blob/main/Trading_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Note: two people can't code on here at the same time it will fuck everything up, so txt the gc if ur going on this

In [1]:
import pandas as pd
import numpy as np
from typing import Tuple
import yfinance as yf
import time
from scipy.signal import argrelextrema
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def calculate_macd(df: pd.DataFrame) -> pd.DataFrame:
  """
  Calculates MACD given our data frame, assuming there is a close and macd
  column. Returns the updated data frame.
  """

  #Can technically change slow and fast periods, but not recommended.
  fast_period:int = 12
  slow_period:int = 26

  #Calculate ema using a given function
  ema_fast = df['Close'].ewm(span=fast_period, adjust=False).mean()
  ema_slow = df['Close'].ewm(span=slow_period, adjust=False).mean()

  #Calc macd
  df['Macd'] = ema_fast - ema_slow
  print(df.columns)
  return df

"""
stock = yf.Ticker("MCD") #change stuff in quotes here to change tag
data = stock.history(period="1mo")
data.to_csv("MCD_stocks.csv")
df = pd.read_csv("MCD_stocks.csv")
df = df.drop(['Dividends', 'Stock Splits'], axis=1)
df['Macd'] = np.nan
"""

In [ ]:
def is_local_minima(series: pd.Series, window: int = 10) -> pd.Series:
    """
    Helper func. Identify local minima in a series using rolling window.
    """
    #return (series == series.rolling(window=window, center=True).min())

    minima_idx = argrelextrema(series.values, comparator=np.less, order=window)[0]
    local_min = pd.Series(False, index=series.index)
    local_min.iloc[minima_idx] = True
    return local_min

   # local_min = pd.Series(index=series.index, data=False)
    #For all points in series, check if it is the least point above and below a
    #window
   # for i in range(window, len(series) - window):
    #  before_window = series.iloc[i-window:i]
   #   after_window = series.iloc[i+1:i+window+1]
   #   before_condition = (series.iloc[i] <= before_window).mean() >= 0.8
   #   after_condition = (series.iloc[i] <= after_window).mean() >= 0.8
        #if all(series.iloc[i] <= series.iloc[i-window:i]) and \
          # all(series.iloc[i] <= series.iloc[i+1:i+window+1]):
          #  local_min.iloc[i] = True
    #  if before_condition and after_condition:
        #  local_min.iloc[i] = True

   # return local_min

def detect_bullish_div(df: pd.DataFrame, window:int = 10) -> pd.Series:
  """
  Dectect bullish divergence with given MACD and price. Returns a series that
  contains divergence points.
  """
  #series is basically a dict but eaiser to use bc its from pandas lib

  price_minima = find_local_minima(df['Close'], window=10)
  macd_minima = find_local_minima(df['Macd'], window=10)
  divergence = pd.Series(index=df.index, data=False)
  print("MACD Local min Found:", macd_minima.sum())
  print("Bullish Minima:", find_local_minima(df['Close']).sum())
  #Scans for div
  for i in range(window, len(df) - window):
        #Check if there was a macd min at our current point
        if macd_minima.iloc[i]:
            #Check previous macd min points
            for j in range(i - window, i):
                tolerance:float = 0.1
                if df['Macd'].iloc[i] > df['Macd'].iloc[j] * (1 - tolerance) and df['Close'].iloc[i] < df['Close'].iloc[j] * (1 + tolerance):
                    #Check if satisfies div conditions: MACD making a higher low
                    #and close getting lower
                    divergence.iloc[i] = True
                break
  return divergence
"""
calculate_macd(df)
detect_bullish_div(df, window=10)
df.head()
"""

In [ ]:
def find_local_maxima(series: pd.Series, window: int = 10) -> pd.Series:
    """
    Helper func. Identify local maxima in a series using window. Returns series
    of local maxima.
    """
    #return (series == series.rolling(window=window, center=True).max())
    #local_max = pd.Series(index=series.index, data=False)
    maxima_idx = argrelextrema(series.values, comparator=np.greater, order=window)[0]
    local_max = pd.Series(False, index=series.index)
    local_max.iloc[maxima_idx] = True
    return local_max
    #for i in range(window, len(series) - window):
        #if all(series.iloc[i] >= series.iloc[i-window:i]) and \
          # all(series.iloc[i] >= series.iloc[i+1:i+window+1]):
           # local_max.iloc[i] = True
        # Consider it a max if it's higher than most (80%) of surrounding point,
        #else it is too narrow of a range
      #  before_window = series.iloc[i-window:i]
      #  after_window = series.iloc[i+1:i+window+1]
     #   before_condition = (series.iloc[i] >= before_window).mean() >= 0.8
     #   after_condition = (series.iloc[i] >= after_window).mean() >= 0.8
      #  if before_condition and after_condition:
       #     local_max.iloc[i] = True
    #return local_max


def detect_bearish_div(df: pd.DataFrame, window:int= 10) -> pd.Series:
  """
  Dectect bearish divergence with given MACD and price. Returns a series that
  contains divergence points.
  """

  price_maxima = find_local_maxima(df['Close'], window=10)
  macd_maxima = find_local_maxima(df['Macd'], window=10)
  print("MACD Local max Found:", macd_maxima.sum())
  divergence = pd.Series(index=df.index, data=False)
  print("Bearish Maxima:", find_local_maxima(df['Close']).sum())
  #Scans for div
  for i in range(window, len(df) - window):
      if macd_maxima.iloc[i]:
          # Look back for a previous maximum
          #for j in range(i-1, max(i-window, 0), -1):
          for j in range(i-window, i):
              #if macd_maxima.iloc[j]:
              tolerance:float = 0.1
              if (df['Macd'].iloc[i] < df['Macd'].iloc[j] * (1 - tolerance)
              and df['Close'].iloc[i] > df['Close'].iloc[j] * (1 + tolerance)):
              #if macd_maxima.iloc[j] and df['Macd'].iloc[i] < df['Macd'].iloc[j]:
                  #Check if satisfies div conditions: MACD making a lower low
                  #and close getting higher
                 # if (df['Macd'].iloc[i] < df['Macd'].iloc[j] and
                    #  df['Close'].iloc[i] > df['Close'].iloc[j]):
                    #  divergence.iloc[i] = True
                  #if df['Close'].iloc[i] > df['Close'].iloc[j]:
                  print("bearish detected")
                  divergence.iloc[i] = True
              break

  return divergence
"""
calculate_macd(df)
detect_bearish_div(df, window=10)
df.head()
"""

In [ ]:
def total_strat(df: pd.DataFrame, investment: float = 10000, stop_loss: float = 0.05, take_profit: float = 0.1) -> pd.DataFrame:

  #Make copy of df to edit and return, do NOT edit the df directly
  df = df.copy()

  #Get the stuff
  df = calculate_macd(df)
  bullish_div = detect_bullish_div(df, window=20)
  bearish_div = detect_bearish_div(df,window=20)
  print("Bullish Div Count:", bullish_div.sum())
  print("Bearish Div Count:", bearish_div.sum())

  # Initialize trading variables
  df['position'] = 0
  df['portfolio_value'] = investment
  position = 0 # -1 short, 0 neutral, 1 long
  entry_price = 0

  for i in range(1, len(df)):
    current_price = df['Close'].iloc[i]
    prev_price = df['Close'].iloc[i-1]

    #Exit conditions
    if position != 0:
      price_change = (current_price - entry_price) / entry_price

      # Exit conditions for long position
      if position == 1:
        if price_change <= -stop_loss or price_change >= take_profit:
          position = 0
          df.loc[df.index[i], 'position'] = 0

      # Exit conditions for short position
      elif position == -1:
        if price_change >= stop_loss or price_change <= -take_profit:
            position = 0
            df.loc[df.index[i], 'position'] = 0

    #Entry conditions, basically div is gonna tell you if you want long or short
    elif position == 0:
      if bullish_div.iloc[i] and not bearish_div.iloc[i]:
        position = 1
        entry_price = current_price
        df.loc[df.index[i], 'position'] = 1

      elif bearish_div.iloc[i] and not bullish_div.iloc[i]:
        position = -1
        entry_price = current_price
        df.loc[df.index[i], 'position'] = -1

    df.loc[df.index[i], 'position'] = position
    #Updates portfolio value based on position
    if position == 0:
        df.loc[df.index[i], 'portfolio_value'] = df['portfolio_value'].iloc[i-1]
        #print(f"Day {i}: Position={position}, Portfolio={df['portfolio_value'].iloc[i]}")
    else:
        if position == 1:
          returns = (current_price - prev_price) / prev_price
        else:
          returns = (prev_price - current_price) / prev_price
        df.loc[df.index[i], 'portfolio_value'] = df['portfolio_value'].iloc[i-1] * (1 + returns)

        '''
        price_change = df['Close'].iloc[i] / df['Close'].iloc[i-1] - 1
        if position == -1:
           price_change = (entry_price - current_price) / entry_price
            #price_change = -price_change
        df.loc[df.index[i], 'portfolio_value'] = \
            df['portfolio_value'].iloc[i-1] * (1 + price_change)
        '''
  df['Bullish Divergence'] = bullish_div
  df['Bearish Divergence'] = bearish_div
  return df


In [ ]:
#Initalize date. NOTE: It must have a macd column and close column denoted "Close" and "Macd", pandas is case sensitive!

#McDonalds
df = pd.read_csv('/content/drive/My Drive/MCD.csv')
#stock = yf.Ticker("OMGA") change stuff in quotes here to change tag
#data = stock.history(period="5y")
#data.to_csv("XHLD.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.rename(columns={'Close/Last': 'Close'}, inplace=True)
df['Close'] = df['Close'].apply(lambda x: float(x[1:]))
df.head(50)

print(df.columns)
print(df)
print(df.index)
print(df.index[3])

In [ ]:
#df = df.drop(['Dividends', 'Stock Splits'], axis=1)
df['Macd'] = np.nan
calculate_macd(df)
result:pd.DataFrame = total_strat(df,investment=1000,stop_loss=0.05, take_profit=0.1)


#first_trade_index = result[result['position'] != 0].index[0]
print("\nRows around first trade:")
#print(result.loc[(result.index >= first_trade_index - pd.Timedelta(days=2)) &
                 #(result.index <= first_trade_index + pd.Timedelta(days=2)),
                 #['Close', 'position', 'portfolio_value']])

# Let's also look at the overall performance
print("\nFirst 10 rows:")
print(result[['Close', 'position', 'portfolio_value']].head(10))

print("\nLast 10 rows:")
print(result[['Close', 'position', 'portfolio_value']].tail(10))

# Print some statistics
print("\nStrategy Statistics:")
print(f"Number of trades: {len(result[result['position'] != result['position'].shift(1)].dropna())}")
print(f"Initial portfolio value: {result['portfolio_value'].iloc[0]:.2f}")
print(f"Final portfolio value: {result['portfolio_value'].iloc[-1]:.2f}")


#XXX